# library

In [1]:
from platform import python_version
print(python_version())
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import csv
import matplotlib.pyplot as plt

import scipy as sp   # 과학 계산용 파이썬 모듈
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import json
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

3.8.12


# 크롤링 및 텍스트 분석

* 필터를 변경하여 기간, 키워드, 출력 파일 이름을 변경 가능 
* maxTweets를 10개

In [2]:
fname = '한국행정구역분류_2022.1.1.기준.xlsx'
data_region = pd.read_excel(fname, sheet_name = [1], skiprows=2, dtype={'대분류':str, '중분류':str, '소분류':str})
data_region[1].drop('Unnamed: 0', axis=1, inplace=True)
data_citydo_all = data_region[1]['시도'][1:]
print(data_citydo_all)
data_citydo_unique = data_citydo_all.unique()
booleanArray1 = [not pd.isnull(number) for number in data_citydo_unique]
data_citydo_unique_notna = data_citydo_unique[booleanArray1]
print(data_citydo_unique_notna)
data_citygungu_all = data_region[1]['시군구'][1:]
data_citygungu_unique = data_citygungu_all.unique()
idx_region=0

1         서울특별시
2         서울특별시
3         서울특별시
4         서울특별시
5         서울특별시
         ...   
3801    제주특별자치도
3802    제주특별자치도
3803    제주특별자치도
3804    제주특별자치도
3805    제주특별자치도
Name: 시도, Length: 3805, dtype: object
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '세종특별자치시' '경기도'
 '강원도' '충청북도' '충청남도' '전라북도' '전라남도' '경상북도' '경상남도' '제주특별자치도']


In [3]:
se_citydo = pd.Series(data_citydo_unique_notna)
df_citydo = se_citydo.to_frame()
df_citydo.columns=['시도']

In [43]:
###날짜붙여넣기
maxTweets = 10
flag_save = 1
maxpage = 1 #input("최대 크롤링할 페이지 수 입력하시오: ") #10,20...
sort = '0' # input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
sta_date = '2019.05.20' #2019.01.04
end_date = '2021.09.30'
date_range = range(0,10)

#각 크롤링 결과 저장하기 위한 리스트 선언
title_text=[]
link_text=[]
source_text=[]
contents_text=[]
result={}

#엑셀로 저장하기 위한 변수
#RESULT_PATH ='C:/'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#내용 정제화 함수
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)
    
df_citydo_found = df_citydo.copy()
#df_citydo_found['지표'] = 0

for idx_date in date_range:
    idx_region=0
    i=0
    idx_num=[1,2,3,4,5]
    list_region = ['인천']
    #list_region = data_citydo_unique_notna  #=========================임시
    list_keyword = ['수돗물 적수']

    s_date = (datetime.strptime(sta_date.replace(".","-"), '%Y-%m-%d') + relativedelta(days = 10*(idx_date))).strftime('%Y-%m-%d')
    e_date = (datetime.strptime(sta_date.replace(".","-"), '%Y-%m-%d') + relativedelta(days = 10*(idx_date+1))).strftime('%Y-%m-%d')
    print('s_date:',s_date)
    
    df_citydo_found[s_date,'~',e_date]=0
    
    for region in list_region:
        title_text = []
        link_text = []
        source_text = []
        contents_text=[]

        for keyword in list_keyword:
            query = region +' ' + keyword
            #print(query)
            e_to = e_date.replace("-","")
            s_from = s_date.replace("-","")
            
            # 트위터
           
            csvFile = open('pfe_tweets_result.csv', 'a', newline='', encoding='utf8')
            #Use csv writer
            csvWriter = csv.writer(csvFile)
            csvWriter.writerow(['id','date','tweet',])

            # Write tweets into the csv file,  lang:en   lang:ko
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' lang:ko since:'+s_date+' until:'+e_date+' -filter:links -filter:replies').get_items()):
                    
                if i > maxTweets :
                    break  
                csvWriter.writerow([tweet.id, tweet.date, tweet.content])
                df_tweet = pd.DataFrame([tweet.content])
                #print('df_tweet', df_tweet)


                if i == 0:
                    tweet_cum = df_tweet
                else:
                    tweet_cum = pd.concat([tweet_cum, df_tweet])


            
            print('tweet_cum', tweet_cum)
            
            df = tweet_cum

            list_keyword_full = ['수돗물 적수']
            list_search = ' '.join(region) + ' ' + ' '.join(list_keyword_full)
            dfNew = pd.DataFrame([list_search])

            if len(df) < 1:
                pass


            else:
                df_tfidf_base = pd.concat([df,dfNew])

                list_tfidf = df_tfidf_base.values.tolist()
                list_tfidf_str = [str (item) for item in list_tfidf]
                corpus = list_tfidf_str
                tfidfv = TfidfVectorizer().fit(corpus)
                tfidfv_array = tfidfv.transform(corpus).toarray()


                cos_sim_df = pd.DataFrame(cosine_similarity(tfidfv_array, tfidfv_array))
                len_contents = len(cos_sim_df)-1
                #print('cos_sim_df:',cos_sim_df)

                cos_sim_df_sorted = cos_sim_df.iloc[len_contents].sort_values(ascending=False)            
                cos_sim_df_sorted_others = cos_sim_df_sorted[1:]
                print('cos_sim_df_sorted_others:',cos_sim_df_sorted_others)

                #len_cos_sim_high_rank = math.floor(len(cos_sim_df_sorted_others) / 3)
                cri_cos_sim_high_rank = 0.1 
                idx_cos_sim_high_rank = cos_sim_df_sorted_others > cri_cos_sim_high_rank
                cos_sim_df_sorted_others_highrank = cos_sim_df_sorted_others[idx_cos_sim_high_rank]
                #cos_sim_df_sorted_others_highrank = cos_sim_df_sorted_others[:len_cos_sim_high_rank]

                print('cos_sim_df_sorted_others_highrank:',cos_sim_df_sorted_others_highrank)
                cos_sim_df_sorted_others_sum = cos_sim_df_sorted_others.sum()/10 * 100 # percent 지표
                #print(cos_sim_df_sorted_others_sum)            

                df_citydo_found.iloc[idx_region, idx_date+1] = cos_sim_df_sorted_others_sum
                #print(s_date,e_date)
        idx_region=idx_region+1 # idx_region 목록 전부 진행할 때 필요



# if flag_save == 1:
#     # 새로 만들 파일이름 지정
#     outputFileName = '%s-%s-%s %s시 %s분 %s초 merging' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
#     df.to_excel(outputFileName+'.xlsx',sheet_name='sheet1')
#     #df.to_csv(outputFileName+'.csv', mode='w')

s_date: 2019-05-20
tweet_cum                                                    0
0  수돗물 적수피해주민들은 자한당에 노후관로 교체를 위한 추경예산안 통과를 강력하게 요...
cos_sim_df_sorted_others: 0    0.072348
Name: 1, dtype: float64
cos_sim_df_sorted_others_highrank: Series([], Name: 1, dtype: float64)
s_date: 2019-05-30
tweet_cum                                                    0
0  인천 서구에 적수 나와서 학교급식도 중단되었고 다들 생수쓰고있다던데 검단쪽은 아닌건...
0               수돗물에 적수가 나온다니 인천이 드디어 진짜 마계가 되는 것인가?
0  인천 서구 수돗물에서 붉은 물이 나오는데 이게 어딜봐서 적합이라는거지..\n심지어 ...
cos_sim_df_sorted_others: 0    0.097154
1    0.000000
2    0.000000
Name: 3, dtype: float64
cos_sim_df_sorted_others_highrank: Series([], Name: 3, dtype: float64)
s_date: 2019-06-09
tweet_cum                                                    0
0  나 인천서구살아서 수돗물때문에 대체급식해봤음. 일주일전에는 빵이나 과일 같은 거 주...
0  좆같은 놈드라 수돗물좀 빨리 돌리도 적수제거기 다느라 거치작 거리고 수압 전나 아프...
0  인천 사는 사람이기도 하고 환경도 전공이라 최근 인천 수돗물 적수사태에 대해서 죠오...
0  우리 동네 적수 땜에 난린데 내 친구 갑자기 그거 땜에 배아프대는데 이새끼 수돗물 ...
cos_sim_df_sorted_others: 3    0

In [44]:
df_citydo_found

,시도,"(2019-05-20, ~, 2019-05-30)","(2019-05-30, ~, 2019-06-09)","(2019-06-09, ~, 2019-06-19)","(2019-06-19, ~, 2019-06-29)","(2019-06-29, ~, 2019-07-09)","(2019-07-09, ~, 2019-07-19)","(2019-07-19, ~, 2019-07-29)","(2019-07-29, ~, 2019-08-08)","(2019-08-08, ~, 2019-08-18)","(2019-08-18, ~, 2019-08-28)"
0,서울특별시,0.723484,0.971545,4.171818,3.19094,3.19094,3.19094,0.723484,0.723484,0.723484,0.723484
1,부산광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
2,대구광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
3,인천광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
4,광주광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
5,대전광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
6,울산광역시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
7,세종특별자치시,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
8,경기도,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
9,강원도,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
